In [6]:
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.mip_solver import mip_solve
from cloudmanufacturing.validation import construct_delta, objvalue

import pandas as pd
from tqdm import trange
import numpy as np

In [7]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")
for problem in dataset:
    print(f'Problem: {problem["name"]}')

100%|██████████| 18/18 [00:00<00:00, 212.60it/s]


Problem: 5,10,10-1
Problem: 5,10,10-2
Problem: 5,10,10-3
Problem: 10,10,10-1
Problem: 10,10,10-2
Problem: 10,10,10-3
Problem: 5,10,20-1
Problem: 5,10,20-2
Problem: 5,10,20-3
Problem: 5,20,10-1
Problem: 5,20,10-2
Problem: 5,20,10-3
Problem: 5,20,20-1
Problem: 5,20,20-2
Problem: 5,20,20-3
Problem: 5,5,5-1
Problem: 5,5,5-2
Problem: 5,5,5-3


In [ ]:
problem = dataset[2]
delta, gamma, status, value = mip_solve(problem)
print(f'Problem {problem["name"]}: {value:.2f}, {status}')

In [ ]:
objvalue(problem, gamma, delta)

In [ ]:
objvalue(problem, gamma, construct_delta(problem, gamma))

## Running all problems

In [2]:
dataset = read_fatahi_dataset("../data/train_data.xlsx")

100%|██████████| 1000/1000 [26:56<00:00,  1.62s/it]


In [8]:
from joblib import Parallel, delayed
from tqdm import trange

def process_problem(i, problem):
    delta, gamma, status, value = mip_solve(problem)

    if status.name == 'OPTIMAL':
        np.save(f'./solutions/delta_{i}_op.npy', delta)
        np.save(f'./solutions/gamma_{i}_op.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    if status.name == 'FEASIBLE':
        np.save(f'./solutions/delta_{i}_fb.npy', delta)
        np.save(f'./solutions/gamma_{i}_fb.npy', gamma)
        return {'i':i,
                'value': value,
                'objvalue': objvalue(problem, gamma, delta),
                'status': status.name}
    
    print(i," wasn't solved")
    return None

In [9]:
sum_info = Parallel(n_jobs=-1)(delayed(process_problem)(i, dataset[i]) for i in trange(len(dataset)))
sum_info = [info for info in sum_info if info is not None]

pd.DataFrame(sum_info).to_csv('./mip_solutions/info.csv', index=False)

100%|██████████| 18/18 [00:00<00:00, 8998.51it/s]


In [5]:
pd.DataFrame(sum_info)

,i,value,objvalue,status
0,0,-5.869061e+08,-5.869061e+08,OPTIMAL
1,1,-2.866272e+09,-2.866272e+09,OPTIMAL
2,2,-2.603758e+09,-2.603758e+09,OPTIMAL
3,3,-1.361310e+09,-1.361310e+09,OPTIMAL
4,4,-2.234574e+09,-2.234574e+09,OPTIMAL
...,...,...,...,...
793,988,-4.517290e+10,-4.517290e+10,FEASIBLE
794,990,-2.343572e+10,-2.343572e+10,FEASIBLE
795,994,-2.567149e+10,-2.567149e+10,OPTIMAL
796,995,-5.411509e+10,-5.411509e+10,OPTIMAL


In [ ]:
# sum_info = []
# for i in trange(len(dataset)):
#     problem = dataset[i]
#     delta, gamma, status, value = mip_solve(problem)
#     if status.name == 'OPTIMAL':
#         np.save(f'./mip_solutions/delta_{i}_opt.npy', delta)
#         np.save(f'./mip_solutions/gamma_{i}_opt.npy', gamma)
#         sum_info.append({'value':value,
#                         'objvalue':objvalue(problem, gamma, delta),
#                         'status':status
#                         })
#     if status.name == 'FEASIBLE':
#         np.save(f'./mip_solutions/delta_{i}_opt.npy', delta)
#         np.save(f'./mip_solutions/gamma_{i}_opt.npy', gamma)
#         sum_info.append({'value':value,
#                         'objvalue':objvalue(problem, gamma, delta),
#                         'status':status
#                         })
# pd.DataFrame(sum_info).to_csv('./mip_solutions/info.csv',index=False)